In [60]:
from argopy import IndexFetcher as ArgoIndexFetcher
import pandas as pd
import numpy as np

index_loader = ArgoIndexFetcher(cache=True)
df=index_loader.region([-180,180,-90.,90.]).to_dataframe()
df.head()

,file,date,longitude,latitude,ocean,date_update,wmo,institution,profiler_type
0,aoml/13858/profiles/R13858_001.nc,1997-07-28 20:26:20,-11.863,-0.126,A,2018-10-11 18:05:56,13858,"AOML, USA","Webb Research, no conductivity"
1,aoml/13857/profiles/R13857_001.nc,1997-07-29 20:03:00,-16.032,0.267,A,2018-10-11 18:05:20,13857,"AOML, USA","Webb Research, no conductivity"
2,aoml/13859/profiles/R13859_001.nc,1997-07-30 14:45:11,-19.688,0.236,A,2018-10-11 18:09:03,13859,"AOML, USA","Webb Research, no conductivity"
3,aoml/31810/profiles/R31810_001.nc,1997-08-01 07:59:00,-27.039,-0.402,A,2018-10-11 18:04:29,31810,"AOML, USA","Webb Research, no conductivity"
4,aoml/31855/profiles/R31855_001.nc,1997-08-02 15:53:47,-34.184,0.429,A,2018-10-11 18:05:03,31855,"AOML, USA","Webb Research, no conductivity"


In [61]:
#extract cycle
import re
df['cycle']=df.file.apply(lambda x: int(re.split(r'[.D]',re.split('_',x)[1])[0]))
#extract months
months=pd.to_datetime(df.date.values).month
df['month']=months
df.head()

,file,date,longitude,latitude,ocean,date_update,wmo,institution,profiler_type,cycle,month
0,aoml/13858/profiles/R13858_001.nc,1997-07-28 20:26:20,-11.863,-0.126,A,2018-10-11 18:05:56,13858,"AOML, USA","Webb Research, no conductivity",1,7
1,aoml/13857/profiles/R13857_001.nc,1997-07-29 20:03:00,-16.032,0.267,A,2018-10-11 18:05:20,13857,"AOML, USA","Webb Research, no conductivity",1,7
2,aoml/13859/profiles/R13859_001.nc,1997-07-30 14:45:11,-19.688,0.236,A,2018-10-11 18:09:03,13859,"AOML, USA","Webb Research, no conductivity",1,7
3,aoml/31810/profiles/R31810_001.nc,1997-08-01 07:59:00,-27.039,-0.402,A,2018-10-11 18:04:29,31810,"AOML, USA","Webb Research, no conductivity",1,8
4,aoml/31855/profiles/R31855_001.nc,1997-08-02 15:53:47,-34.184,0.429,A,2018-10-11 18:05:03,31855,"AOML, USA","Webb Research, no conductivity",1,8


In [62]:
ldat=df.query('cycle == 0').groupby('wmo').last()
ldat.head(5)                   

,file,date,longitude,latitude,ocean,date_update,institution,profiler_type,cycle,month
wmo,,,,,,,,,,
69000,coriolis/69000/profiles/D69000_000.nc,1999-10-03 22:46:00,-22.080,41.940,A,2015-10-19 19:08:18,"Ifremer, France","Provor, no conductivity",0,10
69001,coriolis/69001/profiles/D69001_000.nc,1999-10-04 03:26:00,-18.996,44.972,A,2015-10-19 19:13:40,"Ifremer, France","Provor, no conductivity",0,10
69002,coriolis/69002/profiles/D69002_000.nc,1999-10-04 04:19:00,-21.980,44.310,A,2015-10-19 19:36:22,"Ifremer, France","Provor, no conductivity",0,10
69003,coriolis/69003/profiles/D69003_000.nc,1999-10-11 03:58:00,-17.000,42.110,A,2015-10-19 19:53:48,"Ifremer, France","Provor, no conductivity",0,10
69004,coriolis/69004/profiles/D69004_000.nc,1999-10-11 03:58:00,-18.850,41.840,A,2015-10-19 20:00:56,"Ifremer, France","Provor, no conductivity",0,10


In [63]:
len(ldat)

4230

In [64]:
# For future plot
month_name=['January','February','March','April','May','June','July','August','September','October','November','December']
count_a=np.zeros_like(month_name,dtype=float)
count_i=np.zeros_like(month_name,dtype=float)
count_p=np.zeros_like(month_name,dtype=float)
i=0

for m in np.unique(ldat.month):
    si=ldat.query('month == '+str(m)).groupby(['ocean']).size()
    for j in range(len(si)):
        if si.index[j]=="A":
            count_a[i] = si['A'];
        elif si.index[j]=="P":
            count_p[i] = si['P'];
        elif si.index[j]=="I":
            count_i[i] = si['I'];
    i+=1
    
d = {'month': month_name, 'Atlantic': count_a, 'Pacific': count_p, 'Indian': count_i, 
     'Season': ['Winter','Winter','Winter','Spring','Spring','Spring','Summer','Summer','Summer','Autumn','Autumn','Autumn']}
df = pd.DataFrame(data=d)

df

,month,Atlantic,Pacific,Indian,Season
0,January,123.0,150.0,87.0,Winter
1,February,109.0,223.0,55.0,Winter
2,March,174.0,180.0,36.0,Winter
3,April,115.0,129.0,24.0,Spring
4,May,156.0,161.0,35.0,Spring
5,June,154.0,116.0,18.0,Spring
6,July,106.0,168.0,4.0,Summer
7,August,86.0,166.0,19.0,Summer
8,September,181.0,223.0,28.0,Summer
9,October,142.0,224.0,74.0,Autumn


In [67]:
from bokeh.plotting import figure, show
from bokeh.models import Label
from collections import OrderedDict
from bokeh.io import output_notebook

ocean_color = OrderedDict([("Atlantic","#428bca"),("Pacific", "#2e4045"),("Indian",  "#d9534f"  )])
season_color = OrderedDict([("Winter", "#bae1ff"),("Spring", "#96ceb4"),("Summer", "#fff68f"),("Autumn", "#e69584")])

width = 1000
height = 1000
inner_radius = 90
outer_radius = 300

minr = 0 #np.log10(1.)
maxr = 250 #np.log10(2000)
a = (outer_radius - inner_radius) / (maxr - minr)
b = outer_radius - a * maxr

def rad(mic):
    #return a * np.log10(mic) + b
    return a * mic + b

big_angle = 2.0 * np.pi / (len(df) + 1)
small_angle = big_angle / 7

p = figure(plot_width=width, plot_height=height, title="", x_axis_type=None, y_axis_type=None, 
           x_range=(-420, 420), y_range=(-420, 420),min_border=0, outline_line_color="black",
           background_fill_color="#f1f1f1")

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# annular wedges
angles = np.pi/2 - big_angle/2 - df.index.to_series()*big_angle
colors = [season_color[Season] for Season in df.Season]
p.annular_wedge(0, 0, inner_radius, outer_radius, -big_angle+angles, angles, color=colors)

# small wedges
p.annular_wedge(0, 0, inner_radius, rad(df.Atlantic),-big_angle+angles+5*small_angle,
                -big_angle+angles+6*small_angle,color=ocean_color['Atlantic'])
p.annular_wedge(0, 0, inner_radius, rad(df.Pacific),-big_angle+angles+3*small_angle, 
                -big_angle+angles+4*small_angle, color=ocean_color['Pacific'])
p.annular_wedge(0, 0, inner_radius, rad(df.Indian), -big_angle+angles+1*small_angle,
                -big_angle+angles+2*small_angle, color=ocean_color['Indian'])

# circular axes and lables
#labels = [1,10,100,1000]
labels = [0,50,100,150,200]
radii = rad(np.array(labels))
p.circle(0, 0, radius=radii, fill_color=None, line_color="white")
p.text(0, radii, [str(r) for r in labels], text_font_size="10pt", text_align="center", text_baseline="middle")

# radial axes
p.annular_wedge(0, 0, inner_radius-5, outer_radius+5, -big_angle+angles, -big_angle+angles, color="black")

# DAC labels
xr = (radii[-1]+65)*np.cos(np.array(-big_angle/2 + angles))
yr = (radii[-1]+65)*np.sin(np.array(-big_angle/2 + angles))
label_angle=np.array(-big_angle/2+angles)
label_angle[label_angle < -np.pi/2] += np.pi # easier to read labels on the left side
p.text(xr, yr, df.month, angle=label_angle, text_font_size="9pt", text_align="center", text_baseline="middle")

# OK, these hand drawn legends are pretty clunky, will be improved in future release
p.circle([-60, -60, 30, 30], [-360, -380, -360, -380], color=list(season_color.values()), radius=7,line_color='black')
p.text([-45, -45, 45, 45], [-360, -380, -360, -380], text=[gr for gr in season_color.keys()],
       text_font_size="10pt", text_align="left", text_baseline="middle")
p.rect([-30, -30, -30], [18, 0, -18], width=30, height=13, color=list(ocean_color.values()))
p.text([-5, -5, -5], [18, 0, -18], text=list(ocean_color), text_font_size="10pt", text_align="left", text_baseline="middle")

#TITLE
mytext = Label(x=0, y=350, text='Number of argo floats deployed per month & ocean in 2000-2019',text_font_size="12pt", text_align="center", text_baseline="middle")
p.add_layout(mytext)

output_notebook()

show(p)

Loading BokehJS ...